<!-- $$\text{OL}:V(x,y)=-V_-\cos^2(2k_Lx)\cos^2(2k_Ly)+V_+\left(\sqrt{2\pi}\sigma_+\right)^2e^{-\pi\left(\sqrt{2\pi}\sigma_+\right)^2\left({k_x}^2+{k_y}^2\right)}$$ -->
$$\text{OL}:V(x,y)=-V_-\cos^2(k_Lx)\cos^2(k_Ly)+V_+e^{-\left(\frac{{x}^2+{y}^2}{2\sigma^2}\right)}$$

The sinusoidal component is periodic with frequency $2k_L$ whereas the Gaussian component is periodic with frequency $k_L$. The full potential is, therefore, periodic with frequency $k_L$.

Increasing $V_-$, the depth of the attractive sinusoidal square lattice potential, makes the dispersive bands interpolate from a $2k_L$-separated pair of half-parabolas to the $\pm$-pair of sinusoids with wavelength $4k_L$ and amplitude $2t$ predicted by the tight-binding approximation.

Increasing $V_+$, the amplitude of the repulsive Gaussian block-out potential, flattens the central band.

Increasing $\sigma_+$, the spread of the repulsive Gaussian block-out potential, flattens the central band and increases the separation between the central band the bottom dispersive band.

$$\text{TB}:\;E_\sigma(k_x,k_y)=\sigma2t\sqrt{\cos^2\left(\frac{k_x}{2k_L}\pi\right)+\cos^2\left(\frac{k_y}{2k_L}\pi\right)},\; \sigma\in\{-1,0,+1\}$$

In [ ]:
import numpy as np
import tensorflow as tf

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from plotly.offline import init_notebook_mode
from IPython.display import display, HTML

init_notebook_mode()
display(HTML('<script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/3.2.2/es5/tex-mml-svg.min.js" integrity="sha512-4MXl9OmsJPCU3LySQiKq4baSCSNFha8CBJL7NVSmN+WJJNLhJUycJgGdxJlxAY2ih8jhP9juGYq2ThUihYBKIw==" crossorigin="anonymous" referrerpolicy="no-referrer"></script>'))

In [ ]:
num_quasimomenta = 99
max_momentum_site = 9

max_momentum_site_change = 2 * max_momentum_site
num_momentum_sites = (max_momentum_site_change + 1) ** 2

In [ ]:
quasimomentum = np.mgrid[0:2:num_quasimomenta*1j, 1:1:1j].reshape(2,-1).T # [k_L]

momentum_site = np.mgrid[-max_momentum_site:max_momentum_site+1:1,
                         -max_momentum_site:max_momentum_site+1:1].reshape(2,-1).T # Fundamental: k_L

momentum = 2*momentum_site + quasimomentum[:, np.newaxis]
kinetic = np.linalg.norm(momentum, axis=-1)**2

momentum_site_change = momentum_site[:, np.newaxis] - momentum_site

In [ ]:
def render_momentum_site_potential(attractive_strength, repulsive_strength, repulsive_spread):
    attractive_potential = -1/16 * attractive_strength * np.pad(np.array([
        [1, 0, 2, 0, 1],
        [0, 0, 0, 0, 0],
        [2, 0, 4, 0, 2],
        [0, 0, 0, 0, 0],
        [1, 0, 2, 0, 1]
    ]), max_momentum_site_change - 2)

    momentum_site = np.meshgrid(*(np.arange(-max_momentum_site_change, max_momentum_site_change+1),)*2)
    repulsive_potential = repulsive_strength * (np.sqrt(2*np.pi) * repulsive_spread)**2 * np.exp(-np.pi * (np.sqrt(2*np.pi) * repulsive_spread)**2 * (momentum_site[0]**2 + momentum_site[1]**2))

    return attractive_potential + repulsive_potential

In [ ]:
def compute_spatial_potential(momentum_site_potential, num_points=0):
    padded_momentum_site_potential = np.pad(momentum_site_potential, num_points)

    position = np.fft.fftshift(np.fft.fftfreq(n=padded_momentum_site_potential.shape[0])) # [1/k_L]
    spatial_potential = np.real(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(padded_momentum_site_potential))))

    return position, spatial_potential

In [ ]:
def compute_dispersion(momentum_site_potential):
    potential = momentum_site_potential[
        momentum_site_change[:, :, 0] + max_momentum_site_change,
        momentum_site_change[:, :, 1] + max_momentum_site_change,
    ]

    energy = tf.linalg.diag(kinetic) + potential[np.newaxis, ...]

    dispersion = tf.reshape(tf.linalg.eigvalsh(energy), (num_quasimomenta, num_momentum_sites))

    return dispersion

In [ ]:
def compute_tight_binding_parameters(dispersion):
    width = np.max(dispersion[:, 2]) - np.min(dispersion[:, 2])
    zero = np.max(dispersion[:, 1])
    tunneling = width/2

    return zero, tunneling

In [ ]:
def compute_tight_binding_dispersion(zero, tunneling):
    return np.array([-1, 0, +1])[:, np.newaxis] * 2*tunneling * np.abs(np.cos(quasimomentum[:, 0] * np.pi/2)) + zero

In [ ]:
attractive_strength = 50
repulsive_strength = 2
repulsive_spread = 0.128

momentum_site_potential = render_momentum_site_potential(attractive_strength, repulsive_strength, repulsive_spread)

position, spatial_potential = compute_spatial_potential(momentum_site_potential, num_points=100)
dispersion = compute_dispersion(momentum_site_potential)
zero, tunneling = compute_tight_binding_parameters(dispersion)
tight_binding_dispersion = compute_tight_binding_dispersion(zero, tunneling)

figure = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        rf"$\text{{Potential }} V(x)$",
        rf"$\text{{Dispersion }} E(k_x) \longrightarrow (E_0/E_R={zero:.3f}, t/E_R={tunneling:.3f})$"
    )
)

for band in range(3):
    figure.add_trace(go.Scatter(x=quasimomentum[:, 0], y=dispersion[:, band], name=fr"$\text{{OL}}: {['-', '0', '+'][band]}$"), row=1, col=2)

for band in range(3):
    figure.add_trace(go.Scatter(x=quasimomentum[:, 0], y=tight_binding_dispersion[band], name=fr"$\text{{TB}}: {['-', '0', '+'][band]}$", line=dict(dash='dash')), row=1, col=2)

figure.update_layout(
    xaxis2=dict(title=r"$\text{Quasimomentum }k_x/k_L$"),
    yaxis2=dict(title=r"$\text{Energy }E/E_R$"),
)

figure.add_trace(go.Scatter(x=position, y=spatial_potential[:, spatial_potential.shape[1]//2], name=""), row=1, col=1)
figure.update_layout(
    xaxis=dict(title=r"$\text{Position }xk_L$"),
    yaxis=dict(title=r"$\text{Energy }E/E_R$"),
)

figure.update_layout(
    title_text=fr"$\text{{Analysis @ }}(V_-/E_R={attractive_strength}; V_+/E_R={repulsive_strength}, \sigma_+k_L={repulsive_spread})$",
    autosize=False, width=1500, height=750)

figure.show()

# Attractive Strength

In [ ]:
attractive_strengths = np.linspace(0, 50, 101)
repulsive_strength = 5
repulsive_spread = 0.05

momentum_site_potentials = [ render_momentum_site_potential(attractive_strength, repulsive_strength, repulsive_spread) for attractive_strength in attractive_strengths]
spatial_potentials = [ compute_spatial_potential(momentum_site_potential, num_points=100) for momentum_site_potential in momentum_site_potentials ]
dispersions = [ compute_dispersion(momentum_site_potential) for momentum_site_potential in momentum_site_potentials ]
tight_binding_parameters = [ compute_tight_binding_parameters(dispersion) for dispersion in dispersions ]
tight_binding_dispersions = [ compute_tight_binding_dispersion(*parameters) for parameters in tight_binding_parameters ]

In [ ]:
figure = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        rf"$\text{{Potential }} V(x)$",
        rf"$\text{{Dispersion }} E(k_x)$"
    )
)

for (position, spatial_potential), dispersion, tight_binding_dispersion in zip(spatial_potentials, dispersions, tight_binding_dispersions):
    for band in range(3):
        figure.add_trace(go.Scatter(x=quasimomentum[:, 0], y=dispersion[:, band], 
                                    name=fr"$\text{{OL}}: {['-', '0', '+'][band]}$", 
                                    visible=False), row=1, col=2)
    for band in range(3):
        figure.add_trace(go.Scatter(x=quasimomentum[:, 0], y=tight_binding_dispersion[band],
                                    name=fr"$\text{{TB}}: {['-', '0', '+'][band]}$", 
                                    line=dict(dash='dash'), visible=False), row=1, col=2)
    figure.add_trace(go.Scatter(x=position, y=spatial_potential[:, spatial_potential.shape[1]//2], name="", visible=False), row=1, col=1)

num_step_traces = 7

for i in range(num_step_traces):
    figure.data[i].visible = True

steps = []
for i, (attractive_strength, (zero, tunneling)) in enumerate(zip(attractive_strengths, tight_binding_parameters)):
    step = dict(
        method="update",
        label=f"{attractive_strength:.1f}",
        args=[{"visible": [False] * len(figure.data)},
              {"title": fr"$\text{{Analysis @ }}(V_-/E_R={attractive_strength:.1f}; V_+/E_R={repulsive_strength:.1f}, \sigma_+k_L={repulsive_spread:.1f}) \longrightarrow (E_0/E_R={zero:.3f}, t/E_R={tunneling:.3f})$"},
              ],
    )
    for j in range(num_step_traces):
        step["args"][0]["visible"][num_step_traces*i + j] = True
    steps.append(step)

attractive_strength = attractive_strengths[0]
zero, tunneling = tight_binding_parameters[0]
figure.update_layout(
    title=dict(text=fr"$\text{{Analysis @ }}(V_-/E_R={attractive_strength:.1f}; V_+/E_R={repulsive_strength:.1f}, \sigma_+k_L={repulsive_spread:.1f}) \longrightarrow (E_0/E_R={zero:.3f}, t/E_R={tunneling:.3f})$", font=dict(size=30)),
    xaxis=dict(title=r"$\text{Position }xk_L$"),
    yaxis=dict(title=r"$\text{Energy }E/E_R$"),
    xaxis2=dict(title=r"$\text{Quasimomentum }k_x/k_L$"),
    yaxis2=dict(title=r"$\text{Energy }E/E_R$"),
    sliders=[dict(active=0, steps=steps)],
    autosize=False, width=1500, height=750
)

figure.show()

# Repulsive Strength

In [ ]:
attractive_strength = 50
repulsive_strengths = np.linspace(0, 50, 100)
repulsive_spread = 0.128

momentum_site_potentials = [ render_momentum_site_potential(attractive_strength, repulsive_strength, repulsive_spread) for repulsive_strength in repulsive_strengths]
dispersions = [ compute_dispersion(momentum_site_potential) for momentum_site_potential in momentum_site_potentials ]
tight_binding_parameters = [ compute_tight_binding_parameters(dispersion) for dispersion in dispersions ]
tight_binding_dispersions = [ compute_tight_binding_dispersion(*parameters) for parameters in tight_binding_parameters ]

In [ ]:
show_tight_binding = True

figure = go.Figure()

for dispersion, tight_binding_dispersion in zip(dispersions, tight_binding_dispersions):
    for band in range(3):
        figure.add_trace(go.Scatter(x=quasimomentum[:, 0], y=dispersion[:, band], name=f"OL {band}", visible=False))
    if show_tight_binding:
        for band in range(3):
            figure.add_trace(go.Scatter(x=quasimomentum[:, 0], y=tight_binding_dispersion[band], name=f"TB {band}", line=dict(dash='dash'), visible=False))

num_step_traces = 3 * (2 if show_tight_binding else 1)

for i in range(num_step_traces):
    figure.data[i].visible = True

steps = []
for i, repulsive_strength in enumerate(repulsive_strengths):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(figure.data)},
              {"title": f"Repulsive Strength: {repulsive_strength:.2f}"}],
    )
    for j in range(num_step_traces):
        step["args"][0]["visible"][num_step_traces*i + j] = True
    steps.append(step)

figure.update_layout(
    title="Dispersion",
    sliders=[dict(active=0, steps=steps)],
    autosize=False, width=750, height=750
)

figure.show()

# Repulsive Spread

In [ ]:
attractive_strength = 50
repulsive_strength = 50
repulsive_spreads = np.linspace(0, 0.15, 100)

momentum_site_potentials = [ render_momentum_site_potential(attractive_strength, repulsive_strength, repulsive_spread) for repulsive_spread in repulsive_spreads]
dispersions = [ compute_dispersion(momentum_site_potential) for momentum_site_potential in momentum_site_potentials ]
tight_binding_dispersions = [ compute_tight_binding_dispersion(dispersion) for dispersion in dispersions ]

In [ ]:
show_tight_binding = False

figure = go.Figure()

for dispersion, tight_binding_dispersion in zip(dispersions, tight_binding_dispersions):
    for band in range(3):
        figure.add_trace(go.Scatter(x=quasimomentum[:, 0], y=dispersion[:, band], name=f"OL {band}", visible=False))
    if show_tight_binding:
        for band in range(3):
            figure.add_trace(go.Scatter(x=quasimomentum[:, 0], y=tight_binding_dispersion[band], name=f"TB {band}", line=dict(dash='dash'), visible=False))

num_step_traces = 3 * (2 if show_tight_binding else 1)

for i in range(num_step_traces):
    figure.data[i].visible = True

steps = []
for i, repulsive_spread in enumerate(repulsive_spreads):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(figure.data)},
              {"title": f"Repulsive Spread: {repulsive_spread:.2f}"}],
    )
    for j in range(num_step_traces):
        step["args"][0]["visible"][num_step_traces*i + j] = True
    steps.append(step)

figure.update_layout(
    title="Dispersion",
    sliders=[dict(active=0, steps=steps)],
    autosize=False, width=750, height=750
)

figure.show()